In [1]:
# import all necessary packages & modules
import os
import time
import numpy as np
import pandas as pd
import json
import pyomo.environ as pyomo
from pyomo.opt import ProblemFormat
import matplotlib.pyplot as plt
import networkx as nx
import highspy


import good_model_working
from good_model_working.reload import deep_reload

#Change this to point to your version of cbc or use another solver
solver_name='cbc'

In [2]:
# import the model data & sets
def load_data(filepath):        

# Load the pickled file
    with open(filepath, 'r') as file:
        # Load the JSON data into a Python dictionary
        data = json.load(file)
    return data

model_data = load_data('/Users/peterambiel/Desktop/good_model/Model Input/all_input_objects.json')
model_sets = load_data('/Users/peterambiel/Desktop/good_model/Model Input/all_input_sets_sorted.json')

In [3]:
# create graph
graph = nx.node_link_graph(model_data, directed = True, multigraph = False)

In [ ]:
# import good_model_working
deep_reload(good_model_working)

# run model 
problem = good_model_working.opt_model.Opt_Model(graph, model_sets)

In [ ]:
# solve model
solver = pyomo.SolverFactory('appsi_highs')
solution = solver.solve(problem.model, tee=True)